In [1]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn import metrics
import seaborn as sns
import shutil
import datetime

In [2]:
data_path = 'data.csv'
df = pd.read_csv(data_path)

In [3]:
df.head()

,Layer-1,Layer-2,Ana-1,Ana-2,Thickness,WaveLength,Core_Real,CoreImag,TotalPower,CorePower,EffectiveArea,EML,CLoss
0,Au,TiO2,1.33,1.33,40,1500.0,1.4327,-3.380000e-09,5.240000e-10,5.230000e-10,1.940000e-11,19.9,0.12289
1,Au,TiO2,1.33,1.33,40,1520.0,1.4322,-4.400000e-09,5.030000e-10,5.010000e-10,1.660000e-11,19.9,0.15782
2,Au,TiO2,1.33,1.33,40,1540.0,1.4316,-5.840000e-09,4.960000e-10,4.940000e-10,1.570000e-11,19.9,0.20700
3,Au,TiO2,1.33,1.33,40,1560.0,1.4311,-8.070000e-09,4.930000e-10,4.920000e-10,1.650000e-11,19.9,0.28230
4,Au,TiO2,1.33,1.33,40,1580.0,1.4305,-1.190000e-08,4.630000e-10,4.620000e-10,1.990000e-11,19.9,0.41038


In [4]:
df.shape

(5527, 13)

In [5]:
dummy_fields = ["Layer-1","Layer-2"]

for each in dummy_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
    df = pd.concat([df,dummies], axis=1)

In [6]:
fields_to_drop = ["Layer-1","Layer-2"]
df = df.drop(fields_to_drop, axis=1)

In [7]:
df.head()

,Ana-1,Ana-2,Thickness,WaveLength,Core_Real,CoreImag,TotalPower,CorePower,EffectiveArea,EML,CLoss,Layer-1_AU,Layer-1_Au,Layer-1_Au+Ta2O5,Layer-1_TiO2,Layer-2_Au,Layer-2_TIO2,Layer-2_Ta2O5,Layer-2_TiO2
0,1.33,1.33,40,1500.0,1.4327,-3.380000e-09,5.240000e-10,5.230000e-10,1.940000e-11,19.9,0.12289,0,1,0,0,0,0,0,1
1,1.33,1.33,40,1520.0,1.4322,-4.400000e-09,5.030000e-10,5.010000e-10,1.660000e-11,19.9,0.15782,0,1,0,0,0,0,0,1
2,1.33,1.33,40,1540.0,1.4316,-5.840000e-09,4.960000e-10,4.940000e-10,1.570000e-11,19.9,0.20700,0,1,0,0,0,0,0,1
3,1.33,1.33,40,1560.0,1.4311,-8.070000e-09,4.930000e-10,4.920000e-10,1.650000e-11,19.9,0.28230,0,1,0,0,0,0,0,1
4,1.33,1.33,40,1580.0,1.4305,-1.190000e-08,4.630000e-10,4.620000e-10,1.990000e-11,19.9,0.41038,0,1,0,0,0,0,0,1


In [8]:
for each in df.columns:
  print(each,' '*(20-len(each)), df[each].isnull().values.any())

Ana-1                 False
Ana-2                 False
Thickness             False
WaveLength            False
Core_Real             False
CoreImag              False
TotalPower            False
CorePower             False
EffectiveArea         False
EML                   False
CLoss                 False
Layer-1_AU            False
Layer-1_Au            False
Layer-1_Au+Ta2O5      False
Layer-1_TiO2          False
Layer-2_Au            False
Layer-2_TIO2          False
Layer-2_Ta2O5         False
Layer-2_TiO2          False


In [9]:
tar = ['Core_Real', 'CoreImag', 'TotalPower', 'CorePower','EML', 'CLoss']
# tar = ['CLoss']
x = df[["Layer-1_Au",	"Layer-1_Au+Ta2O5",	"Layer-1_TiO2",	"Layer-2_Au",	"Layer-2_Ta2O5",	"Layer-2_TiO2","Ana-1","Ana-2","Thickness","WaveLength"]]
y = df[tar]

In [10]:
x.head()

,Layer-1_Au,Layer-1_Au+Ta2O5,Layer-1_TiO2,Layer-2_Au,Layer-2_Ta2O5,Layer-2_TiO2,Ana-1,Ana-2,Thickness,WaveLength
0,1,0,0,0,0,1,1.33,1.33,40,1500.0
1,1,0,0,0,0,1,1.33,1.33,40,1520.0
2,1,0,0,0,0,1,1.33,1.33,40,1540.0
3,1,0,0,0,0,1,1.33,1.33,40,1560.0
4,1,0,0,0,0,1,1.33,1.33,40,1580.0


In [11]:
quant_features = tar
scaled_features = {}

for each in quant_features:
    mean, std = y[each].mean(), y[each].std()
    scaled_features[each] = [mean, std]
    y.loc[:, each] = (y[each] - mean)/std 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6392\3167035514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[:, each] = (y[each] - mean)/std
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6392\3167035514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[:, each] = (y[each] - mean)/std
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6392\3167035514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=45)

In [13]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [14]:
X_train[0:5,:]

array([[1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00,
        1.35e+00, 1.34e+00, 4.00e+01, 1.56e+03],
       [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        1.30e+00, 1.30e+00, 3.50e+01, 1.74e+03],
       [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        1.30e+00, 1.34e+00, 3.50e+01, 2.00e+03],
       [1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00,
        1.36e+00, 1.39e+00, 4.00e+01, 1.60e+03],
       [1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00,
        1.37e+00, 1.33e+00, 4.00e+01, 1.94e+03]])

In [15]:
y_train[0:5,:]

array([[ 0.93622996,  0.31624331,  0.66717191,  0.67116436,  0.70299499,
        -0.32842529],
       [ 0.79642073,  0.31085113,  0.28582925,  0.29048972, -0.78102361,
        -0.32158693],
       [-0.74148081,  0.22358526, -0.59291687, -0.59499259, -0.78102361,
        -0.21952878],
       [ 0.79642073,  0.31502497,  0.52624093,  0.52220471,  0.61938831,
        -0.32675162],
       [-0.48728221,  0.2515663 , -0.61778704, -0.6198192 , -0.38389187,
        -0.2503372 ]])

In [16]:
def get_model():
    model = Sequential()
    model.add(Dense(256, kernel_initializer='normal', input_dim = 10, activation='relu'))
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='linear'))
    return model

In [18]:
# # schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
# #         [300, 14000,45000], [0.001, 0.0001,0.00001,0.000001])
# ada = optimizers.Adam(
#     learning_rate=0.001,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     name="Adam"
# )

# model.compile(loss='mse', optimizer=ada, metrics=['mae', 'mse'])
# model.summary()

In [ ]:

# ada = optimizers.Adagrad(
#     learning_rate=0.01,
#     initial_accumulator_value=0.1,
#     epsilon=1e-07,
#     name='Adagrad'
# )


# ada = tf.keras.optimizers.RMSprop(
#     learning_rate=0.0001,
#     rho=0.9,
#     momentum=0.0,
#     epsilon=1e-07,
#     centered=False,
#     name='RMSprop'
# )


# ada = optimizers.Adadelta(
#     learning_rate=0.01,
#     rho=0.95,
#     epsilon=1e-07,
#     name='Adadelta'
# )

# tf.keras.optimizers.Adamax(
#     learning_rate=0.0001,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     name='Adamax'
# )

In [27]:
# tf_callbacks = tf.keras.callbacks.TensorBoard(log_dir = "logs/fit" , histogram_freq = 1)

In [42]:
datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

'20221116-004146'

In [17]:
!rm -rf ./logs/ 
!mkdir ./logs/

'rm' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.


In [43]:
lr = [3e-3, 3e-4, 3e-5]
tm = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
for i in lr:
    train_step = test_step = 0
    tf_callbacks = tf.keras.callbacks.TensorBoard(log_dir = "logs/"+tm+'/'+str(i) , histogram_freq = 1)
    model = get_model()

    opt = optimizers.Adam(
        learning_rate=i,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam")

    model.compile(loss='mse', optimizer=opt, metrics=['mae', 'mse'])

    history = model.fit(
        X_train, 
        y_train, 
        epochs=500, 
        batch_size=64, 
        validation_split = 0.2,
        callbacks=tf_callbacks)

Epoch 1/500
56/56 [==============================] - 1s 4ms/step - loss: 0.9854 - mae: 0.6644 - mse: 0.9854 - val_loss: 1.0246 - val_mae: 0.6861 - val_mse: 1.0246
Epoch 2/500
56/56 [==============================] - 0s 2ms/step - loss: 0.9563 - mae: 0.6598 - mse: 0.9563 - val_loss: 0.9784 - val_mae: 0.6178 - val_mse: 0.9784
Epoch 3/500
56/56 [==============================] - 0s 2ms/step - loss: 0.8322 - mae: 0.6088 - mse: 0.8322 - val_loss: 0.7497 - val_mae: 0.5350 - val_mse: 0.7497
Epoch 4/500
56/56 [==============================] - 0s 2ms/step - loss: 0.6583 - mae: 0.5170 - mse: 0.6583 - val_loss: 0.6780 - val_mae: 0.5440 - val_mse: 0.6780
Epoch 5/500
56/56 [==============================] - 0s 2ms/step - loss: 0.6129 - mae: 0.5143 - mse: 0.6129 - val_loss: 0.5419 - val_mae: 0.4382 - val_mse: 0.5419
Epoch 6/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4956 - mae: 0.4620 - mse: 0.4956 - val_loss: 0.6817 - val_mae: 0.4981 - val_mse: 0.6817
Epoch 7/500
56/56 [===

56/56 [==============================] - 0s 2ms/step - loss: 0.1341 - mae: 0.1906 - mse: 0.1341 - val_loss: 0.1808 - val_mae: 0.2329 - val_mse: 0.1808
Epoch 52/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1279 - mae: 0.1839 - mse: 0.1279 - val_loss: 0.1663 - val_mae: 0.1609 - val_mse: 0.1663
Epoch 53/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1233 - mae: 0.1755 - mse: 0.1233 - val_loss: 0.1823 - val_mae: 0.2165 - val_mse: 0.1823
Epoch 54/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1253 - mae: 0.1834 - mse: 0.1253 - val_loss: 0.2117 - val_mae: 0.2506 - val_mse: 0.2117
Epoch 55/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1494 - mae: 0.2128 - mse: 0.1494 - val_loss: 0.1767 - val_mae: 0.2022 - val_mse: 0.1767
Epoch 56/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1345 - mae: 0.1914 - mse: 0.1345 - val_loss: 0.1667 - val_mae: 0.1683 - val_mse: 0.1667
Epoch 57/500
56/56 [=========

56/56 [==============================] - 0s 2ms/step - loss: 0.1223 - mae: 0.1872 - mse: 0.1223 - val_loss: 0.2058 - val_mae: 0.2548 - val_mse: 0.2058
Epoch 102/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1202 - mae: 0.1842 - mse: 0.1202 - val_loss: 0.1505 - val_mae: 0.1693 - val_mse: 0.1505
Epoch 103/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1420 - mae: 0.2072 - mse: 0.1420 - val_loss: 0.1906 - val_mae: 0.2531 - val_mse: 0.1906
Epoch 104/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1172 - mae: 0.1775 - mse: 0.1172 - val_loss: 0.1532 - val_mae: 0.1770 - val_mse: 0.1532
Epoch 105/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1057 - mae: 0.1649 - mse: 0.1057 - val_loss: 0.1750 - val_mae: 0.2421 - val_mse: 0.1750
Epoch 106/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1244 - mae: 0.1900 - mse: 0.1244 - val_loss: 0.1426 - val_mae: 0.1606 - val_mse: 0.1426
Epoch 107/500
56/56 [===

Epoch 151/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1087 - mae: 0.1676 - mse: 0.1087 - val_loss: 0.1529 - val_mae: 0.1845 - val_mse: 0.1529
Epoch 152/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1186 - mae: 0.1811 - mse: 0.1186 - val_loss: 0.1378 - val_mae: 0.1790 - val_mse: 0.1378
Epoch 153/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1202 - mae: 0.1858 - mse: 0.1202 - val_loss: 0.1416 - val_mae: 0.1643 - val_mse: 0.1416
Epoch 154/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1018 - mae: 0.1597 - mse: 0.1018 - val_loss: 0.1405 - val_mae: 0.1586 - val_mse: 0.1405
Epoch 155/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1084 - mae: 0.1676 - mse: 0.1084 - val_loss: 0.1393 - val_mae: 0.1763 - val_mse: 0.1393
Epoch 156/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1024 - mae: 0.1653 - mse: 0.1024 - val_loss: 0.1292 - val_mae: 0.1517 - val_mse: 0.1292
Epoch 157/

Epoch 201/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1025 - mae: 0.1671 - mse: 0.1025 - val_loss: 0.1441 - val_mae: 0.1862 - val_mse: 0.1441
Epoch 202/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1008 - mae: 0.1599 - mse: 0.1008 - val_loss: 0.1490 - val_mae: 0.1698 - val_mse: 0.1490
Epoch 203/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1042 - mae: 0.1670 - mse: 0.1042 - val_loss: 0.1415 - val_mae: 0.1705 - val_mse: 0.1415
Epoch 204/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1006 - mae: 0.1632 - mse: 0.1006 - val_loss: 0.1609 - val_mae: 0.1852 - val_mse: 0.1609
Epoch 205/500
56/56 [==============================] - 0s 2ms/step - loss: 0.1023 - mae: 0.1639 - mse: 0.1023 - val_loss: 0.1392 - val_mae: 0.1704 - val_mse: 0.1392
Epoch 206/500
56/56 [==============================] - 0s 2ms/step - loss: 0.0984 - mae: 0.1611 - mse: 0.0984 - val_loss: 0.1279 - val_mae: 0.1534 - val_mse: 0.1279
Epoch 207/

...

Epoch 251/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4984 - mae: 0.4392 - mse: 0.4984 - val_loss: 0.5487 - val_mae: 0.4315 - val_mse: 0.5487
Epoch 252/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4907 - mae: 0.4329 - mse: 0.4907 - val_loss: 0.5335 - val_mae: 0.4308 - val_mse: 0.5335
Epoch 253/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4901 - mae: 0.4308 - mse: 0.4901 - val_loss: 0.5362 - val_mae: 0.4315 - val_mse: 0.5362
Epoch 254/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4976 - mae: 0.4396 - mse: 0.4976 - val_loss: 0.5357 - val_mae: 0.4255 - val_mse: 0.5357
Epoch 255/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4892 - mae: 0.4324 - mse: 0.4892 - val_loss: 0.5327 - val_mae: 0.4291 - val_mse: 0.5327
Epoch 256/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4910 - mae: 0.4340 - mse: 0.4910 - val_loss: 0.5369 - val_mae: 0.4225 - val_mse: 0.5369
Epoch 257/

Epoch 301/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4796 - mae: 0.4294 - mse: 0.4796 - val_loss: 0.5100 - val_mae: 0.4083 - val_mse: 0.5100
Epoch 302/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4684 - mae: 0.4165 - mse: 0.4684 - val_loss: 0.5380 - val_mae: 0.4198 - val_mse: 0.5380
Epoch 303/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4724 - mae: 0.4196 - mse: 0.4724 - val_loss: 0.5251 - val_mae: 0.4594 - val_mse: 0.5251
Epoch 304/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4701 - mae: 0.4191 - mse: 0.4701 - val_loss: 0.5109 - val_mae: 0.4332 - val_mse: 0.5109
Epoch 305/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4758 - mae: 0.4241 - mse: 0.4758 - val_loss: 0.5055 - val_mae: 0.4247 - val_mse: 0.5055
Epoch 306/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4663 - mae: 0.4146 - mse: 0.4663 - val_loss: 0.5065 - val_mae: 0.4256 - val_mse: 0.5065
Epoch 307/

Epoch 351/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4451 - mae: 0.4014 - mse: 0.4451 - val_loss: 0.4909 - val_mae: 0.3903 - val_mse: 0.4909
Epoch 352/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4490 - mae: 0.4046 - mse: 0.4490 - val_loss: 0.4814 - val_mae: 0.3982 - val_mse: 0.4814
Epoch 353/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4473 - mae: 0.4029 - mse: 0.4473 - val_loss: 0.4899 - val_mae: 0.4275 - val_mse: 0.4899
Epoch 354/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4440 - mae: 0.4003 - mse: 0.4440 - val_loss: 0.4805 - val_mae: 0.3972 - val_mse: 0.4805
Epoch 355/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4492 - mae: 0.4025 - mse: 0.4492 - val_loss: 0.5490 - val_mae: 0.4984 - val_mse: 0.5490
Epoch 356/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4540 - mae: 0.4115 - mse: 0.4540 - val_loss: 0.4820 - val_mae: 0.3936 - val_mse: 0.4820
Epoch 357/

Epoch 401/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4204 - mae: 0.3850 - mse: 0.4204 - val_loss: 0.4896 - val_mae: 0.3854 - val_mse: 0.4896
Epoch 402/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4245 - mae: 0.3868 - mse: 0.4245 - val_loss: 0.4851 - val_mae: 0.3884 - val_mse: 0.4851
Epoch 403/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4297 - mae: 0.3921 - mse: 0.4297 - val_loss: 0.4782 - val_mae: 0.4281 - val_mse: 0.4782
Epoch 404/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4202 - mae: 0.3840 - mse: 0.4202 - val_loss: 0.4552 - val_mae: 0.3840 - val_mse: 0.4552
Epoch 405/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4236 - mae: 0.3854 - mse: 0.4236 - val_loss: 0.4577 - val_mae: 0.3903 - val_mse: 0.4577
Epoch 406/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4221 - mae: 0.3856 - mse: 0.4221 - val_loss: 0.4711 - val_mae: 0.3758 - val_mse: 0.4711
Epoch 407/

Epoch 451/500
56/56 [==============================] - 0s 2ms/step - loss: 0.4004 - mae: 0.3752 - mse: 0.4004 - val_loss: 0.4436 - val_mae: 0.3595 - val_mse: 0.4436
Epoch 452/500
56/56 [==============================] - 0s 2ms/step - loss: 0.3962 - mae: 0.3692 - mse: 0.3962 - val_loss: 0.4412 - val_mae: 0.3982 - val_mse: 0.4412
Epoch 453/500
56/56 [==============================] - 0s 2ms/step - loss: 0.3991 - mae: 0.3730 - mse: 0.3991 - val_loss: 0.4507 - val_mae: 0.4114 - val_mse: 0.4507
Epoch 454/500
56/56 [==============================] - 0s 2ms/step - loss: 0.3957 - mae: 0.3710 - mse: 0.3957 - val_loss: 0.4333 - val_mae: 0.3843 - val_mse: 0.4333
Epoch 455/500
56/56 [==============================] - 0s 2ms/step - loss: 0.3933 - mae: 0.3691 - mse: 0.3933 - val_loss: 0.4333 - val_mae: 0.3641 - val_mse: 0.4333
Epoch 456/500
56/56 [==============================] - 0s 2ms/step - loss: 0.3998 - mae: 0.3722 - mse: 0.3998 - val_loss: 0.4353 - val_mae: 0.3889 - val_mse: 0.4353
Epoch 457/

In [27]:
# tf_callbacks = tf.summary.create_file_writer("logs/"+str(lr))
# model = get_model()

# opt = optimizers.Adam(
#     learning_rate=0.01,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     name="Adam")

# model.compile(loss='mse', optimizer=opt, metrics=['mae', 'mse'])

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=500, 
#     batch_size=64, 
#     validation_split = 0.2)

In [46]:
%reload_ext tensorboard
%tensorboard --logdir logs/20221116-004531

In [45]:
tm

'20221116-004531'

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error')
  plt.plot(hist['epoch'], hist['mae'], label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'], label='Val Error')
  plt.legend()
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean mse Error')
  plt.plot(hist['epoch'], hist['mse'], label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'], label='Val Error')
  plt.legend()

plot_history(history)  


In [ ]:
pre = model.predict(X_test)
y = y_test
# k = 2
# pre = pre[0]
# y = y[0]

cnt=0
for i in range(len(pre)):
#       print(y[i],'\t', pre[i],'\t', abs(y[i] - pre[i]))
      cnt = cnt + abs(y[i] - pre[i])
print('Abs Error :', cnt)
print('ERROR :', metrics.mean_absolute_error(y,pre))


# for k in range(6):
#     for i in range(len(pre)):
#     #   print(y[i][k],'\t', pre[i][k],'\t', abs(y[i][k] - pre[i][k]))
#       cnt = cnt + abs(y[i][k] - pre[i][k])
#     print('Abs Error :', cnt)  

In [ ]:
for l in range(6):
    print(tar[l], metrics.r2_score(y[:,l], pre[:,l]))

    plt.figure(figsize=(10,10))
    sns.regplot(x=y[:,l], y=pre[:,l])

In [ ]:
y